In [1]:
import geemap
import ee

from ipyleaflet import basemaps
import datetime

# Indonesian Atmosphere

PM2.5 Data from Copernicus Atmosphere Monitoring Service overlaid with Coal Power Plants data from World Resource Institute (WRI). Units in ug/m^3 and values below 30ug/m^3 are not shown on the map.

In [2]:
now = datetime.datetime.utcnow()
step = int(now.hour / 6)
obs = datetime.datetime(now.year, now.month, now.day, step * 6)

print('Datetime :', obs.strftime("%Y-%m-%d %H:%M UTC"))

Datetime : 2021-07-25 06:00 UTC


In [3]:
Map = geemap.Map(center=(0,120), zoom=5,
    min_zoom=5, max_zoom=12,
    basemap=basemaps.CartoDB.DarkMatter,
    add_google_map=False
)

In [5]:
fname = f'pm2p5_{obs.strftime("%Y%m%dT%H")}.tif'

pm2p5 = ee.Image.loadGeoTIFF(f'gs://cams-neonet/{fname}')
data = pm2p5.resample().reproject(ee.Projection('EPSG:4326'), scale=1000)
data = data.updateMask(data.gt(30))

indo_power = ee.FeatureCollection('WRI/GPPD/power_plants') \
    .filter(ee.Filter.eq('country', 'IDN')) \
    .filter(ee.Filter.eq('fuel1', 'Coal'))

In [7]:
bbox = ee.Geometry.BBox(80, -15, 160, 15)

vmin = 0
vmax = 250
palette = ['blue', 'yellow', 'red', 'black']

vis_params = {
    'min' : vmin,
    'max' : vmax,
    'palette' : palette,
    'opacity' : 0.5,
}

Map.addLayer(data.clip(bbox), vis_params, 'PM2.5')
Map.addLayer(indo_power, {'color' : 'ff0000'}, 'Coal Power Plant')
Map.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax,
    layer_name='PM2.5')
Map.addLayerControl()
Map

Map(bottom=4457.0, center=[-2.67968661580376, 120.41015625000001], controls=(WidgetControl(options=['position'…